## single tree 

better version of tree.ipynb

In [ ]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics    # contains all the effectivness idexes 
import pandas as pd    # for the dataset
from matplotlib import pyplot as plt
import numpy as np

#### importing and preparing the dataset

In [ ]:
labels = ['class', 'spec_num', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']

df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)

# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column

In [ ]:
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

#### finding best hyper-parameters

In [ ]:
# names of all effectivness indexes available in sklearn

# note that roc_auc doesn't work in cv because it is a multiclass classification (we need to specify
# ovo or ovr) and that all roc_auc variants don't work in loocv because the testing sets contain
# only one observation

print(metrics.get_scorer_names())

In [ ]:
# using grid search with k-fold CV to find the best hyperparameters and fitting the tree

# se refit = 'blabla', alla fine viene rifittato l'albero su tutto il dataset scegliendo i parametri
# migliori in base all'indice di accuracy 'blabla'

k = 5

grid_param = {'criterion': ['gini', 'entropy'], 'min_samples_split': np.array(range(2, 8))}
t_cv = GridSearchCV(tree.DecisionTreeClassifier(), grid_param, cv=k, scoring=('accuracy', 'roc_auc_ovo', 'roc_auc_ovr'), refit=False)
t_cv.fit(x, y)

display(pd.DataFrame(t_cv.cv_results_))

In [ ]:
# using now grid search with loocv

grid_param = {'criterion': ['gini', 'entropy'], 'min_samples_split': np.array(range(2, 8))}
t_loocv = GridSearchCV(tree.DecisionTreeClassifier(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
t_loocv.fit(X, y)
print(t_loocv.best_params_)
print(t_loocv.best_score_)

#### fitting the trees

In [ ]:
t_cv_instance = t_cv.best_estimator_
t_loocv_instance = t_loocv.best_estimator_

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(t_cv_instance, filled=True)

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(t_loocv_instance, filled=True)

In [ ]:
print(type(t_cv_instance))
print(type(t_cv))